In [ ]:
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

In [1]:
# Check for input leakage
def check_data_leakage(prices, predictions):
    """
    Check if predictions are aligned with current prices and not future prices.

    Parameters:
    prices (list or array): List of actual prices.
    predictions (list or array): List of predicted values.
    """
    if len(predictions) != len(prices):
        print("Data misalignment: Prices and Predictions lengths differ.")
    else:
        print("Data alignment check passed.")

    shifted_prices = pd.Series(prices).shift(-1)
    correlations = pd.DataFrame({
        'predictions': predictions[:-1],
        'future_price_change': shifted_prices.diff().fillna(0)[:-1]
    }).corr()
    print("Correlation between predictions and future price changes:")
    print(correlations)

In [ ]:
# Feature importance analysis
def analyze_feature_importance(features, target):
    """
    Analyze feature importance using RandomForestClassifier and permutation importance.

    Parameters:
    features (DataFrame): Features input_data.
    target (Series): Target variable.
    """
    clf = RandomForestClassifier(random_state=42)
    clf.fit(features, target)

    importances = pd.DataFrame({
        'Feature': features.columns,
        'Importance': clf.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("Feature Importances:")
    print(importances)

    perm_importance = permutation_importance(clf, features, target, random_state=42)
    perm_importances = pd.DataFrame({
        'Feature': features.columns,
        'Importance': perm_importance.importances_mean
    }).sort_values(by='Importance', ascending=False)
    print("Permutation Importances:")
    print(perm_importances)

In [ ]:
# Risk-Reward Dynamics
def calculate_sharpe_ratio(profits, initial_capital, risk_free_rate=0.01):
    """
    Calculate the Sharpe ratio.

    Parameters:
    profits (array): Array of profit/loss values.
    initial_capital (float): Initial capital invested.
    risk_free_rate (float): Risk-free rate of return.

    Returns:
    float: Sharpe ratio.
    """
    returns = profits / initial_capital
    excess_returns = returns - risk_free_rate
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio

def evaluate_risk_reward(total_capital_history, initial_capital):
    """
    Evaluate risk-reward dynamics by calculating profit distribution and Sharpe ratio.

    Parameters:
    total_capital_history (array): Array of total capital over time.
    initial_capital (float): Initial capital invested.
    """
    profits = np.diff(total_capital_history)
    profit_series = pd.Series(profits)
    print("Profit/Loss Distribution:")
    print(profit_series.describe())

    sharpe_ratio = calculate_sharpe_ratio(profit_series, initial_capital)
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

In [2]:
# Trading Logic Validation
def random_strategy(prices, initial_capital=10000, risk_level=0.1):
    """
    Simulate a random trading strategy.

    Parameters:
    prices (list or array): List of prices.
    initial_capital (float): Starting USD balance.
    risk_level (float): Percentage of capital to risk per trade.

    Returns:
    list: Total capital history over the period.
    """
    usd_balance = initial_capital
    btc_balance = 0
    total_capital_history = []

    for price in prices:
        action = random.choice(['buy', 'sell', 'hold'])
        if action == 'buy' and usd_balance > 0:
            amount_to_invest = usd_balance * risk_level
            btc_bought = amount_to_invest / price
            btc_balance += btc_bought
            usd_balance -= amount_to_invest
        elif action == 'sell' and btc_balance > 0:
            btc_sold = btc_balance * risk_level
            usd_gained = btc_sold * price
            btc_balance -= btc_sold
            usd_balance += usd_gained
        total = usd_balance + btc_balance * price
        total_capital_history.append(total)

    return total_capital_history

In [3]:
# Sensitivity Analysis
def sensitivity_analysis(trading_data, initial_balances, fees):
    """
    Perform sensitivity analysis on trading strategy.

    Parameters:
    trading_data (DataFrame): Data including 'Action', 'Trade_Percentage', and 'prices'.
    initial_balances (list): List of (USD balance, BTC balance) tuples.
    fees (list): List of (maker_fee, taker_fee) tuples.

    Returns:
    DataFrame: Summary of results for each configuration.
    """
    results = []

    for usd_start, btc_start in initial_balances:
        for maker_fee, taker_fee in fees:
            # Assuming process_trades is defined elsewhere
            usd_balance, btc_balance, total_capital_history, _, _, buy_count, sell_count = process_trades(
                trading_data=trading_data,
                usd_balance=usd_start,
                btc_balance=btc_start,
                maker_fee=maker_fee,
                taker_fee=taker_fee
            )

            final_btc_price = trading_data['prices'].iloc[-1]
            remaining_btc_value = btc_balance * final_btc_price
            total_portfolio_value = usd_balance + remaining_btc_value
            profit_loss_percentage = ((total_portfolio_value - usd_start) / usd_start) * 100

            results.append({
                'Initial USD Balance': usd_start,
                'Initial BTC Balance': btc_start,
                'Maker Fee': maker_fee,
                'Taker Fee': taker_fee,
                'Final USD Balance': usd_balance,
                'Final BTC Balance': btc_balance,
                'Total Portfolio Value (USD)': total_portfolio_value,
                'Profit/Loss (%)': profit_loss_percentage,
                'Buy Trades': buy_count,
                'Sell Trades': sell_count
            })

    return pd.DataFrame(results)

In [ ]:
print("Data Leakage Check\n")
check_data_leakage(prices, predictions)

In [ ]:
print("\n\nFeature Importance Analysis\n")
analyze_feature_importance(features, target)

In [ ]:
print("\n\nRisk-Reward Dynamics\n")
evaluate_risk_reward(total_capital_history, initial_capital)

In [ ]:
print("\n\nTrading Logic Validation\n")
random_capital_history = random_strategy(prices, initial_capital=10000, risk_level=0.1)
evaluate_risk_reward(random_capital_history, initial_capital=10000)

In [ ]:
# Run sensitivity analysis
# Initial balances to test
initial_balances = [(10000.0, 0.0), (20000.0, 0.0), (5000.0, 0.5)]

# Fee configurations to test
fees = [(0.0025, 0.0040), (0.0015, 0.0030), (0.0050, 0.0075)]

sensitivity_results = sensitivity_analysis(
    trading_data=data,
    initial_balances=initial_balances,
    fees=fees
)

In [ ]:
print("\n\nSensitivity Analysis\n")
display(sensitivity_results)

In [ ]:
plt.figure(figsize=(12, 6))
for usd_start, btc_start in initial_balances:
    subset = sensitivity_results[sensitivity_results['Initial USD Balance'] == usd_start]
    plt.plot(subset['Maker Fee'], subset['Profit/Loss (%)'], marker='o', label=f'USD: {usd_start}, BTC: {btc_start}')

plt.title('Sensitivity Analysis: Profit/Loss vs Maker Fee', fontsize=14)
plt.xlabel('Maker Fee', fontsize=12)
plt.ylabel('Profit/Loss (%)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()